# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, mean_squared_error, accuracy_score, roc_auc_score, roc_curve
import numpy as np
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install sweetviz -q
!pip install pandas_profiling==1.4.1 -q
!pip install pandas==0.25.3 -q
!pip install -U scikit-learn -q

In [3]:
# id — уникальный идентификатор скважины
# f0, f1, f2 — три признака точек
# product — объём запасов в скважине (тыс. баррелей)
import pandas_profiling

df = pd.set_option('display.float_format', '{:,.2f}'.format)

geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
pandas_profiling.ProfileReport(geo_data_0)

Number of variables,5
Number of observations,100000
Total Missing (%),0.0%
Total size in memory,3.8 MiB
Average record size in memory,40.0 B
Numeric,4
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [5]:
# Проверим на пропуски и дубликаты
df = [geo_data_0, geo_data_1, geo_data_2]

for geo_data in df:
    print('--------------------------')
    print(geo_data.info())
    print('--------------------------')
    print(geo_data.shape)
    print('--------------------------')
    print(geo_data.duplicated().sum())
    print('--------------------------')
    print(geo_data.isna().sum())
    print('--------------------------')
    print(geo_data.describe())

--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
--------------------------
(100000, 5)
--------------------------
0
--------------------------
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
--------------------------
              f0         f1         f2    product
count 100,000.00 100,000.00 100,000.00 100,000.00
mean        0.50       0.25       2.50      92.50
std         0.87       0.50       3.25      44.29
min        -1.41      -0.85     -12.09       0.00
25%        -0.07      -0.20       0.29      56.50
50%         0.50       0.25       2.52      91.85
75%         1.07       0.70       4.72     128.56
max         2.36       1.34      

In [6]:
# Проведем масштабирование количественных признаков

def data_scaler(geo_data):
    features = geo_data.drop(['product', 'id'], axis=1)
    target = geo_data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                                  random_state=12345)
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid

In [7]:
# Разделим на признаки и целевой признак
features_train_0, features_valid_0, target_train_0, target_valid_0 = data_scaler(df[0])
features_train_1, features_valid_1, target_train_1, target_valid_1 = data_scaler(df[1])
features_train_2, features_valid_2, target_train_2, target_valid_2 = data_scaler(df[2])

In [8]:
print(features_train_0.shape, features_valid_0.shape)
print(target_train_0.shape, target_valid_0.shape)

(75000, 3) (25000, 3)
(75000,) (25000,)


In [9]:
features_train_0.describe()

,f0,f1,f2
count,"75,000.00","75,000.00","75,000.00"
mean,0.00,-0.00,0.00
std,1.00,1.00,1.00
min,-2.19,-2.18,-3.89
25%,-0.66,-0.89,-0.68
50%,0.00,-0.00,0.00
75%,0.66,0.89,0.68
max,2.14,2.17,4.15


## Обучение и проверка модели

In [10]:
# Обучим модель LinearRegression

def model_linear(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = pd.Series(model.predict(features_valid))
    
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    mean_pred = predictions_valid.mean()
    
    return predictions_valid, rmse, mean_pred

In [11]:
predictions_valid_0, rmse_0, mean_pred_0 = model_linear(features_train_0, features_valid_0, target_train_0, target_valid_0)
predictions_valid_1, rmse_1, mean_pred_1 = model_linear(features_train_1, features_valid_1, target_train_1, target_valid_1)
predictions_valid_2, rmse_2, mean_pred_2 = model_linear(features_train_2, features_valid_2, target_train_2, target_valid_2)

In [12]:
print(f'Для региона №1 средний запас предсказанного сырья составил {mean_pred_0:.2f}, RMSE модели {rmse_0:.2f}')
print(f'Для региона №2 средний запас предсказанного сырья составил {mean_pred_1:.2f}, RMSE модели {rmse_1:.2f}')
print(f'Для региона №3 средний запас предсказанного сырья составил {mean_pred_2:.2f}, RMSE модели {rmse_2:.2f}')

Для региона №1 средний запас предсказанного сырья составил 92.59, RMSE модели 37.58
Для региона №2 средний запас предсказанного сырья составил 68.73, RMSE модели 0.89
Для региона №3 средний запас предсказанного сырья составил 94.97, RMSE модели 40.03


## Подготовка к расчёту прибыли

###  Расчет достаточного объема сырья для безубыточной разработки новой скважины

In [13]:
Bootstrap_Samples = 1000
Explore = 500             # количество точек для исследования
Mining = 200              # количество точек для разработки
Mining_Budget = 10000    # бюджет на разработку скважин в регионе в млн. руб.
Revenue_Barrel = 0.45     # доход с 1 тыс баррель в млн. руб
Loss_Probability = 0.025  # максимальная вероятность убытков

In [14]:
# средние затраты на разработку одной скважины
price_one_borehole = Mining_Budget / Mining

In [15]:
# количество в тыс. баррель необходимо для покрытия затрат на одну скважину
product_volume = price_one_borehole / Revenue_Barrel

In [16]:
print(f'Необходимый объем сырья для безубыточной разработки 1 скважины в тыс. баррелей: {product_volume:.2f}')

Необходимый объем сырья для безубыточной разработки 1 скважины в тыс. баррелей: 111.11


## Расчёт прибыли и рисков 

In [17]:
# Функция поиска 200 скважин с максимальным объемом нефти
def total_income(target, predictions):
    predict_search = predictions.sort_values(ascending=False).head(200) 
    target_search = target.reset_index(drop=True)[predict_search.index] 
    income = target_search.sum() * Revenue_Barrel 
    return income

In [18]:
print('Прибыль с 200 скважин выбранных по предсказаниям модели составит:')
print(f'для региона №1 - {total_income(target_valid_0, predictions_valid_0)- Mining_Budget:.3f} млн. руб.')
print(f'для региона №2 - {total_income(target_valid_1, predictions_valid_1)- Mining_Budget:.3f} млн. руб.')
print(f'для региона №3 - {total_income(target_valid_2, predictions_valid_2)- Mining_Budget:.3f} млн. руб.')

Прибыль с 200 скважин выбранных по предсказаниям модели составит:
для региона №1 - 3320.826 млн. руб.
для региона №2 - 2415.087 млн. руб.
для региона №3 - 2710.350 млн. руб.


In [19]:
# Функция поиска 500 скважин с максимальным объемом нефти
def revenue(target, predictions, mining):
    predict_search = predictions.sort_values(ascending=False)
    target_search = target[predict_search.index][:Mining]
    income = target_search.sum() * Revenue_Barrel
    return income

In [20]:
# Функция распределения прибыли с помощью bootstrap
def income_region(target, predictions, bootstrap_samples, explore):
    state = np.random.RandomState(12345)
    count = 0
    values = []
    for i in range(Bootstrap_Samples):
        target_subsample = target.reset_index(drop=True).sample(Explore, random_state=state, replace=True)
        probs_sample = predictions[target_subsample.index]
        total_revenue = revenue(target_subsample, probs_sample, Mining) - Mining_Budget
        values.append(total_revenue)
        if total_revenue < 0:
            count += 1
    values = pd.Series(values)
    mean_revenue = values.mean()
    print(f'Средняя прибыль:, {mean_revenue:.2f}', 'млн. рублей')
    
    confidence_interval = st.t.interval(0.95, len(values)-1, mean_revenue, np.std(values, ddof=1))
    print('95%-ый доверительный интервал:', confidence_interval)
    
    pvalue = 1 * count / Bootstrap_Samples
    if pvalue < Loss_Probability:
        print(f'Вероятность убытков равна {pvalue:.2%} и является меньше допустимой, регион подходит по критериям')
    else:
        print(f'Вероятность убытков равна {pvalue:.2%} и является больше допустимой, регион не подходит по критериям')
    
    # левая граница доверительного интервала
    x1, y1 = [confidence_interval[0], confidence_interval[0]], [0, 30]
    # правая граница доверительного интервала
    x2, y2 = [confidence_interval[1], confidence_interval[1]], [0, 30]
    plt.figure()
    x = plt.hist(values,  bins=100)
    plt.plot(x1, y1, x2, y2, marker = 'o')
    plt.title('Гистограмма распределения прибыли')
    plt.xlabel('Прибыль в млн. рублей')
    plt.show()

In [21]:
print('Для региона №1 при случайном выборе 500 скважин')
income_region(target_valid_0, predictions_valid_0, Bootstrap_Samples, Explore)

Для региона №1 при случайном выборе 500 скважин
Средняя прибыль:, 425.94 млн. рублей
95%-ый доверительный интервал: (-118.1730815867329, 970.0501354079183)
Вероятность убытков равна 6.00% и является больше допустимой, регион не подходит по критериям


In [22]:
print('Для региона №2 при случайном выборе 500 скважин')
income_region(target_valid_1, predictions_valid_1, Bootstrap_Samples, Explore)

Для региона №2 при случайном выборе 500 скважин
Средняя прибыль:, 515.22 млн. рублей
95%-ый доверительный интервал: (85.1119884768645, 945.3335584097158)
Вероятность убытков равна 1.00% и является меньше допустимой, регион подходит по критериям


In [23]:
print('Для региона №3 при случайном выборе 500 скважин')
income_region(target_valid_2, predictions_valid_2, Bootstrap_Samples, Explore)

Для региона №3 при случайном выборе 500 скважин
Средняя прибыль:, 435.01 млн. рублей
95%-ый доверительный интервал: (-120.12349557730545, 990.1402211428172)
Вероятность убытков равна 6.40% и является больше допустимой, регион не подходит по критериям


<div class="alert alert-info"> <b>Комментарий студента:</b> 
    
Вывод:

- Выбрав 500 случайных  скважин для проведения геолоразведки, прибыль падает в среднем в 6 раз, с 3 млрд рублей до 0,5 млрд рублей. 
    
- Все три региона являются прибыльными: регион №1 - 426 млн. рублей, регион №2 - 515 млн. рублей, регион №3 - 435 млн. рублей.
    
- 1-ый и 3-й регионы обладают 6 и 6.4 % вероятностью убытков, поэтому не проходят по критериям установленной бизнес-задачи. 
    
- 2-й регион единственный соответствует условию по минимальной вероятности убытков (левая граница 95-% доверительного интервала - положительная). В связи с чем для дальнейшей разработки заказчику предлагается принять только 2-й регион.
   
- Так как сважин очень много, заказчику рекомендуется включить в бизнес-модель параметр - "объем вложений для проведения геолоразведки одной скважины".
    
</div>